# Chapter 17: Generic Patterns and Best Practices

---

## 17.1 Factory Patterns with Generics

Factory patterns abstract object creation, and when combined with generics, they provide type-safe instantiation without sacrificing flexibility.

### 17.1.1 Abstract Factory Pattern

The Abstract Factory pattern provides an interface for creating families of related objects without specifying their concrete classes.

```typescript
// Abstract product interfaces
interface DatabaseConnection {
  connect(): Promise<void>;
  query<T>(sql: string): Promise<T[]>;
  disconnect(): Promise<void>;
}

interface CacheClient {
  get<T>(key: string): Promise<T | null>;
  set<T>(key: string, value: T, ttl?: number): Promise<void>;
  delete(key: string): Promise<void>;
}

// Abstract factory interface
interface InfrastructureFactory {
  createDatabase(): DatabaseConnection;
  createCache(): CacheClient;
}

// Concrete implementations for Production
class PostgresConnection implements DatabaseConnection {
  async connect(): Promise<void> {
    console.log("Connecting to PostgreSQL...");
  }
  
  async query<T>(sql: string): Promise<T[]> {
    console.log(`Executing: ${sql}`);
    return [];
  }
  
  async disconnect(): Promise<void> {
    console.log("Disconnecting from PostgreSQL...");
  }
}

class RedisClient implements CacheClient {
  async get<T>(key: string): Promise<T | null> {
    console.log(`Getting ${key} from Redis`);
    return null;
  }
  
  async set<T>(key: string, value: T, ttl?: number): Promise<void> {
    console.log(`Setting ${key} in Redis (TTL: ${ttl})`);
  }
  
  async delete(key: string): Promise<void> {
    console.log(`Deleting ${key} from Redis`);
  }
}

// Concrete factory for production
class ProductionFactory implements InfrastructureFactory {
  createDatabase(): DatabaseConnection {
    return new PostgresConnection();
  }
  
  createCache(): CacheClient {
    return new RedisClient();
  }
}

// Concrete implementations for Testing (mocks)
class MockDatabase implements DatabaseConnection {
  private data: Map<string, any[]> = new Map();
  
  async connect(): Promise<void> {
    console.log("Mock DB connected");
  }
  
  async query<T>(sql: string): Promise<T[]> {
    return this.data.get(sql) || [];
  }
  
  async disconnect(): Promise<void> {
    console.log("Mock DB disconnected");
  }
  
  seedData(sql: string, data: any[]): void {
    this.data.set(sql, data);
  }
}

class MockCache implements CacheClient {
  private store: Map<string, any> = new Map();
  
  async get<T>(key: string): Promise<T | null> {
    return this.store.get(key) || null;
  }
  
  async set<T>(key: string, value: T): Promise<void> {
    this.store.set(key, value);
  }
  
  async delete(key: string): Promise<void> {
    this.store.delete(key);
  }
}

// Concrete factory for testing
class TestFactory implements InfrastructureFactory {
  createDatabase(): MockDatabase {
    return new MockDatabase();
  }
  
  createCache(): MockCache {
    return new MockCache();
  }
}

// Generic service using the factory
class UserService {
  constructor(
    private db: DatabaseConnection,
    private cache: CacheClient
  ) {}
  
  async getUser(id: string) {
    // Try cache first
    const cached = await this.cache.get<{ id: string; name: string }>(`user:${id}`);
    if (cached) return cached;
    
    // Fetch from DB
    const users = await this.db.query<{ id: string; name: string }>(
      `SELECT * FROM users WHERE id = '${id}'`
    );
    
    if (users[0]) {
      await this.cache.set(`user:${id}`, users[0], 3600);
    }
    
    return users[0];
  }
}

// Usage
async function initializeApp(factory: InfrastructureFactory) {
  const db = factory.createDatabase();
  const cache = factory.createCache();
  
  await db.connect();
  
  const userService = new UserService(db, cache);
  
  // Use service...
  
  await db.disconnect();
}

// Production usage
initializeApp(new ProductionFactory());

// Testing usage with mocks
async function testUserService() {
  const factory = new TestFactory();
  const mockDb = factory.createDatabase();
  
  // Seed test data
  mockDb.seedData("SELECT * FROM users WHERE id = '1'", [{ id: "1", name: "Test" }]);
  
  await initializeApp(factory);
}
```

### 17.1.2 Generic Factory Functions

Factory functions can be generic to create instances of any class while maintaining type safety.

```typescript
// Generic factory with constructor constraint
type Constructor<T> = new (...args: any[]) => T;

function createInstance<T>(ctor: Constructor<T>, ...args: any[]): T {
  return new ctor(...args);
}

// Usage
class User {
  constructor(public name: string, public age: number) {}
}

class Product {
  constructor(public sku: string, public price: number) {}
}

const user = createInstance(User, "John", 30);
// TypeScript knows user is User
console.log(user.name); // Autocomplete works

const product = createInstance(Product, "SKU123", 99.99);
// TypeScript knows product is Product

// Factory with specific constructor signature
interface Entity {
  id: string;
}

type EntityConstructor<T extends Entity> = new (id: string, data: any) => T;

class Repository<T extends Entity> {
  private items: Map<string, T> = new Map();
  
  constructor(
    private entityClass: EntityConstructor<T>
  ) {}
  
  create(id: string, data: any): T {
    const instance = new this.entityClass(id, data);
    this.items.set(id, instance);
    return instance;
  }
  
  findById(id: string): T | undefined {
    return this.items.get(id);
  }
}

class Order implements Entity {
  constructor(
    public id: string,
    public data: { items: string[]; total: number }
  ) {}
}

class Invoice implements Entity {
  constructor(
    public id: string,
    public data: { amount: number; dueDate: Date }
  ) {}
}

const orderRepo = new Repository(Order);
const invoiceRepo = new Repository(Invoice);

const order = orderRepo.create("ord-1", { items: ["a", "b"], total: 100 });
// TypeScript knows order is Order

const invoice = invoiceRepo.create("inv-1", { amount: 100, dueDate: new Date() });
// TypeScript knows invoice is Invoice
```

---

## 17.2 Repository Pattern with Generics

The Repository pattern abstracts data access logic, providing a clean separation between domain and data mapping layers.

### 17.2.1 Generic Repository Interface

```typescript
// Base entity interface
interface Entity {
  id: string | number;
}

// Query options for flexible searching
interface QueryOptions<T> {
  where?: Partial<T>;
  orderBy?: keyof T;
  order?: "asc" | "desc";
  limit?: number;
  offset?: number;
}

// Repository interface
interface IRepository<T extends Entity> {
  findById(id: T["id"]): Promise<T | null>;
  findAll(options?: QueryOptions<T>): Promise<T[]>;
  findOne(options: QueryOptions<T>): Promise<T | null>;
  create(data: Omit<T, "id">): Promise<T>;
  update(id: T["id"], data: Partial<Omit<T, "id">>): Promise<T>;
  delete(id: T["id"]): Promise<boolean>;
  exists(id: T["id"]): Promise<boolean>;
  count(options?: Pick<QueryOptions<T>, "where">): Promise<number>;
}

// Specification pattern for complex queries
interface Specification<T> {
  isSatisfiedBy(item: T): boolean;
  toQuery(): object;
}

// Generic specification implementation
class AndSpecification<T> implements Specification<T> {
  constructor(private specs: Specification<T>[]) {}
  
  isSatisfiedBy(item: T): boolean {
    return this.specs.every(spec => spec.isSatisfiedBy(item));
  }
  
  toQuery(): object {
    return { $and: this.specs.map(s => s.toQuery()) };
  }
}

// Concrete repository implementation
class InMemoryRepository<T extends Entity> implements IRepository<T> {
  protected items: Map<T["id"], T> = new Map();
  private nextId = 1;
  
  constructor(
    private generateId: () => T["id"] = () => this.nextId++ as T["id"]
  ) {}
  
  async findById(id: T["id"]): Promise<T | null> {
    return this.items.get(id) || null;
  }
  
  async findAll(options: QueryOptions<T> = {}): Promise<T[]> {
    let results = Array.from(this.items.values());
    
    if (options.where) {
        // Filter by partial match
      results = results.filter(item => {
        return Object.entries(options.where!).every(([key, value]) => {
          return (item as any)[key] === value;
        });
      });
    }
    
    if (options.orderBy) {
      const order = options.order === "desc" ? -1 : 1;
      results.sort((a, b) => {
        const aVal = (a as any)[options.orderBy!];
        const bVal = (b as any)[options.orderBy!];
        if (aVal < bVal) return -1 * order;
        if (aVal > bVal) return 1 * order;
        return 0;
      });
    }
    
    if (options.offset) {
      results = results.slice(options.offset);
    }
    
    if (options.limit) {
      results = results.slice(0, options.limit);
    }
    
    return results;
  }
  
  async findOne(options: QueryOptions<T>): Promise<T | null> {
    const results = await this.findAll(options);
    return results[0] || null;
  }
  
  async create(data: Omit<T, "id">): Promise<T> {
    const id = this.generateId();
    const item = { ...data, id } as T;
    this.items.set(id, item);
    return item;
  }
  
  async update(id: T["id"], data: Partial<Omit<T, "id">>): Promise<T> {
    const existing = this.items.get(id);
    if (!existing) {
      throw new Error(`Entity with id ${id} not found`);
    }
    const updated = { ...existing, ...data } as T;
    this.items.set(id, updated);
    return updated;
  }
  
  async delete(id: T["id"]): Promise<boolean> {
    return this.items.delete(id);
  }
  
  async exists(id: T["id"]): Promise<boolean> {
    return this.items.has(id);
  }
  
  async count(options: Pick<QueryOptions<T>, "where"> = {}): Promise<number> {
    if (!options.where) {
      return this.items.size;
    }
    const results = await this.findAll({ where: options.where });
    return results.length;
  }
}

// Concrete entity implementations
interface User extends Entity {
  id: string;
  name: string;
  email: string;
  age: number;
}

interface Product extends Entity {
  id: number;
  title: string;
  price: number;
  inStock: boolean;
}

// Usage
async function demonstrateRepository() {
  const userRepo = new InMemoryRepository<User>(() => `user-${Date.now()}`);
  const productRepo = new InMemoryRepository<Product>(() => Date.now());
  
  // Create users
  const user = await userRepo.create({
    name: "John",
    email: "john@example.com",
    age: 30
  });
  
  // Query with filters
  const adults = await userRepo.findAll({
    where: { age: 30 },
    orderBy: "name",
    limit: 10
  });
  
  // Update
  await userRepo.update(user.id, { age: 31 });
  
  // Products
  const product = await productRepo.create({
    title: "Laptop",
    price: 999,
    inStock: true
  });
  
  const inStock = await productRepo.findAll({
    where: { inStock: true }
  });
}
```

### 17.2.2 Repository with Specifications

**Specification Pattern Implementation:**

```typescript
// Abstract specification
abstract class Specification<T> {
  abstract isSatisfiedBy(candidate: T): boolean;
  abstract toQuery(): object;
  
  and(other: Specification<T>): Specification<T> {
    return new AndSpecification(this, other);
  }
  
  or(other: Specification<T>): Specification<T> {
    return new OrSpecification(this, other);
  }
  
  not(): Specification<T> {
    return new NotSpecification(this);
  }
}

class AndSpecification<T> extends Specification<T> {
  constructor(
    private left: Specification<T>,
    private right: Specification<T>
  ) {
    super();
  }
  
  isSatisfiedBy(candidate: T): boolean {
    return this.left.isSatisfiedBy(candidate) && 
           this.right.isSatisfiedBy(candidate);
  }
  
  toQuery(): object {
    return { $and: [this.left.toQuery(), this.right.toQuery()] };
  }
}

class OrSpecification<T> extends Specification<T> {
  constructor(
    private left: Specification<T>,
    private right: Specification<T>
  ) {
    super();
  }
  
  isSatisfiedBy(candidate: T): boolean {
    return this.left.isSatisfiedBy(candidate) || 
           this.right.isSatisfiedBy(candidate);
  }
  
  toQuery(): object {
    return { $or: [this.left.toQuery(), this.right.toQuery()] };
  }
}

class NotSpecification<T> extends Specification<T> {
  constructor(private spec: Specification<T>) {
    super();
  }
  
  isSatisfiedBy(candidate: T): boolean {
    return !this.spec.isSatisfiedBy(candidate);
  }
  
  toQuery(): object {
    return { $not: this.spec.toQuery() };
  }
}

// Concrete specifications for User
class AdultSpecification extends Specification<User> {
  isSatisfiedBy(user: User): boolean {
    return user.age >= 18;
  }
  
  toQuery(): object {
    return { age: { $gte: 18 } };
  }
}

class EmailDomainSpecification extends Specification<User> {
  constructor(private domain: string) {
    super();
  }
  
  isSatisfiedBy(user: User): boolean {
    return user.email.endsWith(`@${this.domain}`);
  }
  
  toQuery(): object {
    return { email: { $regex: `@${this.domain}$` } };
  }
}

// Repository with specification support
class SpecificationRepository<T extends Entity> extends InMemoryRepository<T> {
  async findBySpecification(spec: Specification<T>): Promise<T[]> {
    const all = Array.from(this.items.values());
    return all.filter(item => spec.isSatisfiedBy(item));
  }
}

// Usage
const userRepo = new SpecificationRepository<User>();

const adultSpec = new AdultSpecification();
const companyEmailSpec = new EmailDomainSpecification("company.com");

// Combine specifications
const adultEmployees = adultSpec.and(companyEmailSpec);

const users = await userRepo.findBySpecification(adultEmployees);
```

---

## 17.3 Builder Pattern with Generics

The Builder pattern constructs complex objects step by step. With generics, we can create type-safe builders that track which properties have been set.

### 17.3.1 Type-Safe Builder

```typescript
// Type-level utility to track set properties
type BuilderState<T, SetKeys extends keyof T> = {
  [K in SetKeys]: T[K];
} & {
  [K in Exclude<keyof T, SetKeys>]?: T[K];
};

// Generic builder class
class ObjectBuilder<T extends object, SetKeys extends keyof T = never> {
  private obj: Partial<T> = {};
  
  // Returns builder with updated type state
  set<K extends keyof T>(key: K, value: T[K]): ObjectBuilder<T, SetKeys | K> {
    (this.obj as any)[key] = value;
    return this as any; // Type cast for builder pattern
  }
  
  // Build when all required keys are set
  build(this: ObjectBuilder<T, keyof T>): T {
    return this.obj as T;
  }
  
  // Build with partial (may be unsafe)
  buildPartial(): Partial<T> {
    return { ...this.obj };
  }
}

// Usage
interface User {
  id: string;
  name: string;
  email: string;
  age: number;
}

const user = new ObjectBuilder<User>()
  .set("id", "123")
  .set("name", "John")
  .set("email", "john@example.com")
  .set("age", 30)
  .build(); // OK - all required properties set

// const incomplete = new ObjectBuilder<User>()
//   .set("id", "123")
//   .build(); // ❌ Error: build() requires all properties

// Fluent builder with method chaining
class QueryBuilder<T extends Entity, Filters = {}> {
  private filters: any = {};
  private options: any = {};
  
  where<K extends keyof T>(
    key: K, 
    value: T[K]
  ): QueryBuilder<T, Filters & { [P in K]: T[K] }> {
    this.filters[key] = value;
    return this as any;
  }
  
  orderBy<K extends keyof T>(
    key: K, 
    direction: "asc" | "desc" = "asc"
  ): this {
    this.options.orderBy = key;
    this.options.direction = direction;
    return this;
  }
  
  limit(count: number): this {
    this.options.limit = count;
    return this;
  }
  
  build(): { filters: Filters; options: any } {
    return {
      filters: this.filters,
      options: this.options
    };
  }
}

// Usage
const query = new QueryBuilder<User>()
  .where("age", 30)
  .where("name", "John")
  .orderBy("email", "desc")
  .limit(10)
  .build();

// query.filters has type { age: number; name: string }
```

### 17.3.2 Step-by-Step Builder

**Builder with Required Steps:**

```typescript
// Step-by-step builder ensuring proper construction order
class RequestBuilder<HasMethod extends boolean = false, HasUrl extends boolean = false> {
  private config: {
    method?: string;
    url?: string;
    headers?: Record<string, string>;
    body?: unknown;
  } = {};
  
  // Method step - returns builder with HasMethod = true
  method<T extends "GET" | "POST" | "PUT" | "DELETE">(
    method: T
  ): RequestBuilder<true, HasUrl> {
    this.config.method = method;
    return this as any;
  }
  
  // URL step - returns builder with HasUrl = true
  url(url: string): RequestBuilder<HasMethod, true> {
    this.config.url = url;
    return this as any;
  }
  
  // Optional steps return same type
  headers(headers: Record<string, string>): this {
    this.config.headers = { ...this.config.headers, ...headers };
    return this;
  }
  
  body<T>(body: T): this {
    this.config.body = body;
    return this;
  }
  
  // Execute requires both method and url
  execute(
    this: RequestBuilder<true, true>
  ): Promise<Response> {
    return fetch(this.config.url!, {
      method: this.config.method,
      headers: this.config.headers,
      body: this.config.body ? JSON.stringify(this.config.body) : undefined
    });
  }
}

// Usage
const request = new RequestBuilder()
  .method("POST")
  .url("/api/users")
  .headers({ "Content-Type": "application/json" })
  .body({ name: "John" })
  .execute(); // OK - method and url are set

// const badRequest = new RequestBuilder()
//   .method("GET")
//   .execute(); // ❌ Error: execute requires url to be set
```

---

## 17.4 Generic Utility Functions

Creating reusable utility functions with generics is a common pattern in TypeScript.

### 17.4.1 Common Generic Utilities

```typescript
// Deep partial - makes all properties optional recursively
type DeepPartial<T> = {
  [P in keyof T]?: T[P] extends object ? DeepPartial<T[P]> : T[P];
};

function deepMerge<T extends object>(target: T, source: DeepPartial<T>): T {
  const result = { ...target };
  
  for (const key in source) {
    if (source[key] !== undefined) {
      if (
        typeof source[key] === "object" && 
        source[key] !== null &&
        !Array.isArray(source[key])
      ) {
        (result as any)[key] = deepMerge(
          (result as any)[key] || {},
          source[key] as any
        );
      } else {
        (result as any)[key] = source[key];
      }
    }
  }
  
  return result;
}

// Type-safe property access with path
function get<T, K extends keyof T>(obj: T, key: K): T[K] {
  return obj[key];
}

// Type-safe groupBy
function groupBy<T, K extends keyof T>(
  array: T[],
  key: K
): Map<T[K], T[]> {
  const map = new Map<T[K], T[]>();
  
  for (const item of array) {
    const groupKey = item[key];
    const group = map.get(groupKey) || [];
    group.push(item);
    map.set(groupKey, group);
  }
  
  return map;
}

// Usage
interface User {
  id: number;
  name: string;
  department: string;
}

const users: User[] = [
  { id: 1, name: "John", department: "Engineering" },
  { id: 2, name: "Jane", department: "Engineering" },
  { id: 3, name: "Bob", department: "Sales" }
];

const byDept = groupBy(users, "department");
// Map<string, User[]> with keys "Engineering", "Sales"

// Type-safe memoization
function memoize<T extends (...args: any[]) => any>(fn: T): T {
  const cache = new Map<string, ReturnType<T>>();
  
  return ((...args: Parameters<T>) => {
    const key = JSON.stringify(args);
    if (cache.has(key)) {
      return cache.get(key)!;
    }
    const result = fn(...args);
    cache.set(key, result);
    return result;
  }) as T;
}

const expensive = memoize((n: number) => {
  console.log("Computing...");
  return n * n;
});

expensive(5); // Computing... 25
expensive(5); // 25 (cached)
```

### 17.4.2 Generic Array Utilities

```typescript
// Unique by key
function uniqueBy<T, K extends keyof T>(array: T[], key: K): T[] {
  const seen = new Set<T[K]>();
  return array.filter(item => {
    if (seen.has(item[key])) return false;
    seen.add(item[key]);
    return true;
  });
}

// Sort by multiple fields
function sortBy<T>(array: T[], ...fields: Array<keyof T>): T[] {
  return [...array].sort((a, b) => {
    for (const field of fields) {
      if (a[field] < b[field]) return -1;
      if (a[field] > b[field]) return 1;
    }
    return 0;
  });
}

// Partition array into two based on predicate
function partition<T>(
  array: T[],
  predicate: (item: T) => boolean
): [T[], T[]] {
  const pass: T[] = [];
  const fail: T[] = [];
  
  for (const item of array) {
    if (predicate(item)) {
      pass.push(item);
    } else {
      fail.push(item);
    }
  }
  
  return [pass, fail];
}

// Usage
const [adults, minors] = partition(users, u => u.age >= 18);
```

---

## 17.5 Performance Considerations

Generics in TypeScript are a compile-time only feature - they don't exist at runtime. However, there are still performance considerations to keep in mind.

### 17.5.1 Compile-Time Performance

```typescript
// Complex generic types can slow down compilation
// Avoid deeply nested conditional types in hot paths

// Good: Simple generic
type Simple<T> = T extends object ? T : never;

// Avoid: Deeply nested conditionals
type Complex<T> = 
  T extends object 
    ? T extends Array<infer U>
      ? U extends object
        ? U extends Date
          ? string
          : number
        : boolean
      : string
    : never;

// Use type aliases to break up complex types
type IsArray<T> = T extends Array<infer U> ? U : never;
type IsObject<T> = T extends object ? T : never;
// Compose smaller types rather than nesting deeply
```

### 17.5.2 Runtime Performance

```typescript
// Generics don't add runtime overhead - they're erased
function identity<T>(x: T): T {
  return x;
}

// Compiles to:
// function identity(x) { return x; }

// But instantiations can create different shapes
class Container<T> {
  value: T;
  constructor(value: T) {
    this.value = value;
  }
}

// Each concrete usage is separate in type system but
// at runtime these are the same class structure
const stringC = new Container("hello");
const numberC = new Container(42);

// Avoid excessive instantiations in performance-critical code
// Use constraints to limit type variations
```

---

## 17.6 Common Generic Pitfalls

### 17.6.1 Overuse of `any`

```typescript
// ❌ Bad: Using any defeats the purpose
function badProcess<T>(data: T): any {
  return data as any;
}

// ✅ Good: Preserve type information
function goodProcess<T>(data: T): T {
  return data;
}

// ❌ Bad: Casting through unknown/any without validation
function unsafeCast<T>(x: unknown): T {
  return x as T; // Dangerous!
}

// ✅ Good: Type guard for safety
function safeCast<T>(x: unknown, guard: (x: unknown) => x is T): T | null {
  return guard(x) ? x : null;
}
```

### 17.6.2 Overly Complex Constraints

```typescript
// ❌ Bad: Too complex, hard to understand
function overConstrained<
  T extends Record<string, any> & { id: string } & { validate(): boolean }
>(obj: T): void {}

// ✅ Good: Use interface for complex constraints
interface ValidatableEntity {
  id: string;
  validate(): boolean;
}

function wellConstrained<T extends ValidatableEntity>(obj: T): void {}
```

### 17.6.3 Forgetting Variance

```typescript
// ❌ Bad: Not considering variance
interface Container<T> {
  get(): T;
  set(value: T): void;
}

// This is invariant - restrictive but safe
// Consider splitting into Producer (covariant) and Consumer (contravariant)

// ✅ Good: Split when appropriate
interface Producer<out T> {
  get(): T;
}

interface Consumer<in T> {
  set(value: T): void;
}
```

---

## 17.7 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we explored advanced generic patterns:

**Key Takeaways:**

1. **Factory Patterns**: Abstract creation of related objects with type safety
2. **Repository Pattern**: Type-safe data access with query flexibility
3. **Builder Pattern**: Step-by-step construction with compile-time tracking
4. **Utility Functions**: Reusable generic helpers for common operations
5. **Performance**: Generics are compile-time only but complex types can slow compilation
6. **Pitfalls**: Avoid overuse of `any`, overly complex constraints, and ignoring variance

### Practical Exercises

**Exercise 1: Generic Factory**

Create a plugin system:

```typescript
// 1. Create a Plugin interface with name, version, initialize(), execute()

// 2. Create a PluginFactory<T extends Plugin> that:
//    - Registers plugin constructors
//    - Creates instances with proper typing
//    - Validates plugin compatibility

// 3. Implement concrete plugins (LoggerPlugin, CachePlugin)

// 4. Demonstrate type-safe plugin creation
```

**Exercise 2: Repository Pattern**

Build a type-safe data layer:

```typescript
// 1. Extend the Repository pattern with:
//    - Pagination support
//    - Transaction support
//    - Caching layer

// 2. Implement MongoRepository<T> that:
//    - Implements IRepository<T>
//    - Adds MongoDB-specific queries
//    - Uses generics for collection types

// 3. Add soft delete functionality with generic constraints
```

**Exercise 3: Type-Safe Builder**

Create a SQL query builder:

```typescript
// 1. Create a QueryBuilder that:
//    - Tracks SELECT, FROM, WHERE, ORDER BY, LIMIT states
//    - Ensures SELECT and FROM are set before executing
//    - Provides type-safe column selection based on table schema

// 2. Support joins with type inference for joined tables

// 3. Add compile-time checking for column existence
```

**Exercise 4: Generic Utilities**

Implement utility library:

```typescript
// 1. Create deepClone<T>(obj: T): T that handles circular references

// 2. Create validate<T>(obj: unknown, schema: Schema<T>): obj is T
//    with runtime validation

// 3. Create diff<T>(old: T, new: T): Partial<T> showing changes

// 4. Create merge<T, U>(a: T, b: U): T & U with conflict detection
```

**Exercise 5: Performance Optimization**

Optimize generic code:

```typescript
// 1. Profile compilation time of deeply nested generics
// 2. Refactor to use simpler type constraints
// 3. Implement lazy evaluation for expensive type computations
// 4. Compare bundle size with and without generics
```

### Additional Resources

- **TypeScript Design Patterns**: https://refactoring.guru/design-patterns/typescript
- **Advanced TypeScript Generics**: https://www.typescriptlang.org/docs/handbook/2/generics.html
- **Builder Pattern**: https://refactoring.guru/design-patterns/builder/typescript/example

---

## Coming Up Next: Chapter 18 - Mapped Types

In the next chapter, we will explore:

- Understanding mapped types and how they work
- Built-in mapped types: Partial, Required, Readonly, Pick, Omit
- Creating custom mapped types
- Key remapping with `as`
- Mapping modifiers: `+`, `-`, `?`, `readonly`
- Homomorphic mapped types
- Practical applications of mapped types

Mapped types are one of TypeScript's most powerful features for type transformation, allowing you to create new types by transforming properties of existing types in a type-safe manner.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='16. generics_in_depth.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../6. advanced_type_features/18. mapped_types.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
